### Краткое описание задачи

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

### Содержание

##### Часть 1.  <a href='#1-bullet'>Подготовка:</a>

##### Часть 2.  <a href='#2-bullet'>Обучение моделей:</a> 

##### Часть 3.  <a href='#3-bullet'>Вывод:</a> 

<a id='1-bullet'></a>

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import CountVectorizer


In [2]:
try:
    df=pd.read_csv(r'C:\Users\Susanna\Downloads\toxic_comments.csv')
except:
    df=pd.read_csv(r'/datasets/toxic_comments.csv')

display(df.head())
df.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Исходные данные содержат 159571 комментариев и их характиристики.  

В тексе есть символы \n. Заменю их на пробелы, тобы они не участвовали в предобработке:

In [3]:
df['text']=df['text'].str.replace('\n', ' ')

Проведу  сначал токенизацию, а затем и лемматизирую всех слов:

In [4]:
import re
def tokenize(text):
    text = re.sub(r'[^a-zA-Z ^0-9]', ' ', str(text))
    return text.split()

In [5]:
def token(row):
    return tokenize(row['text'])

In [6]:
df['tokenized']=df.apply(token, axis=1)

In [7]:
from nltk.stem import WordNetLemmatizer 
m = WordNetLemmatizer()
def func(row):
    lemmatized_output = ' '.join([m.lemmatize(w) for w in row['tokenized']])
    return lemmatized_output

df['lemmatized']=df.apply(func, axis=1)

Посмотрю что получилось:

In [8]:
df.head(10)

,text,toxic,tokenized,lemmatized
0,Explanation Why the edits made under my userna...,0,"[Explanation, Why, the, edits, made, under, my...",Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,"[D, aww, He, matches, this, background, colour...",D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,"[Hey, man, I, m, really, not, trying, to, edit...",Hey man I m really not trying to edit war It s...
3,""" More I can't make any real suggestions on im...",0,"[More, I, can, t, make, any, real, suggestions...",More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,"[You, sir, are, my, hero, Any, chance, you, re...",You sir are my hero Any chance you remember wh...
5,""" Congratulations from me as well, use the to...",0,"[Congratulations, from, me, as, well, use, the...",Congratulations from me a well use the tool we...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,"[COCKSUCKER, BEFORE, YOU, PISS, AROUND, ON, MY...",COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7,Your vandalism to the Matt Shirvington article...,0,"[Your, vandalism, to, the, Matt, Shirvington, ...",Your vandalism to the Matt Shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,"[Sorry, if, the, word, nonsense, was, offensiv...",Sorry if the word nonsense wa offensive to you...
9,alignment on this subject and which are contra...,0,"[alignment, on, this, subject, and, which, are...",alignment on this subject and which are contra...


Разобью исходный датасет на обучающую и тестовую выборки:

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    df.drop(['toxic', 'text', 'tokenized'], axis=1), df.toxic, test_size=0.1, random_state=12345, shuffle=True)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((143613, 1), (15958, 1), (143613,), (15958,))

Укажу стоп-слова:

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Susanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создам матрицы tf_idf для обучающей и тестовой выборок:

In [11]:
features_train_tf_idf = count_tf_idf.fit_transform(features_train['lemmatized'].values.astype('U')) 

In [12]:
features_test_tf_idf = count_tf_idf.transform(features_test['lemmatized'].values.astype('U')) 

In [13]:
features_train_tf_idf.shape, features_test_tf_idf.shape 

((143613, 167052), (15958, 167052))

#### Вывод 1  
Провела токенизацию и лемматизацию, избавилась от стоп-слов, получила матрицы со значениями tf_idf.  
Исходные данные подготовлены.


<a id='2-bullet'></a>

# 2. Обучение моделей

Первой попробую обучить логистическую регрессию:

##### 2.1 Логистическая регрессия

In [14]:
model=LogisticRegression()
model.fit(features_train_tf_idf, target_train)

C:\Users\Susanna\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
predictions=model.predict(features_test_tf_idf)

In [16]:
from sklearn.metrics import f1_score
f1=f1_score(target_test, predictions)
f1

0.7361211247296322

Значение метрики f1 получилось 0,74. Это меньше требуемого по условиям задачи.   

Попробую обучить LGBMClassifier:  

##### 2.2 LGBMClassifier

In [17]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
model = LGBMClassifier(random_state=12345, num_leaves=300)

In [18]:
model.fit(features_train_tf_idf, target_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=300, objective=None,
               random_state=12345, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
predictions=model.predict(features_test_tf_idf)

In [20]:
f1=f1_score(target_test, predictions)
f1

0.7684140676841407

Значение метрики f1 получилось 0,77. Это удовлетворяет условиям задачи.   

#### Вывод 2:  
Модель LGBMClassifier показала результат лучше (f1=0.77), чем модель LogisticRegression (f1=0.74).

<a id='3-bullet'></a>

# 3. Выводы

#### Общий вывод:

Загрузила и подготовила исходные данные.  
Попробовала 2 модели: LogisticRegression и LGBMClassifier.  
LGBMClassifier показала наилучший результат f1=0.77.  